# Starting point for Spark on Google Cloud

In [5]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as F
spark = SparkSession.builder.appName('Graphs-HW2').getOrCreate()

In [6]:
# Load data
a2q_sdf = spark.read.format("com.databricks.spark.csv").option("delimiter", ' ') .load("s3n://upenn-bigdataanalytics/data/sx-stackoverflow-a2q.txt")
c2q_sdf = spark.read.format("com.databricks.spark.csv").option("delimiter", ' ') .load("s3n://upenn-bigdataanalytics/data/sx-stackoverflow-c2q.txt")
c2a_sdf = spark.read.format("com.databricks.spark.csv").option("delimiter", ' ') .load("s3n://upenn-bigdataanalytics/data/sx-stackoverflow-c2a.txt")

Py4JJavaError: An error occurred while calling o27.load.
: java.io.IOException: No FileSystem for scheme: s3n
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:2660)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:2667)
	at org.apache.hadoop.fs.FileSystem.access$200(FileSystem.java:94)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:2703)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:2685)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:373)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:295)
	at org.apache.spark.sql.execution.datasources.DataSource$$anonfun$14.apply(DataSource.scala:350)
	at org.apache.spark.sql.execution.datasources.DataSource$$anonfun$14.apply(DataSource.scala:348)
	at scala.collection.TraversableLike$$anonfun$flatMap$1.apply(TraversableLike.scala:241)
	at scala.collection.TraversableLike$$anonfun$flatMap$1.apply(TraversableLike.scala:241)
	at scala.collection.immutable.Li

## Step 2.3

In [2]:
# TODO: read files, load graph_sdf, etc.

In [3]:
# Add as many cells as you like
graph_sdf = a2q_sdf.union(c2q_sdf)
graph_sdf = graph_sdf.union(c2a_sdf)

graph_sdf = graph_sdf.select(graph_sdf._c0.cast('int').alias('from_node'),\
                            graph_sdf._c1.cast('int').alias('to_node'))

In [ ]:
# Compute nodes_sdf
filteredF_sdf = graph_sdf[graph_sdf.from_node < 8].select(graph_sdf.from_node.alias('node')).distinct()
filteredT_sdf = graph_sdf[graph_sdf.to_node < 8].select(graph_sdf.to_node.alias('node')).distinct()
nodes_sdf = filteredF_sdf.union(filteredT_sdf).distinct().cache()

In [ ]:
def sdf_is_empty(sdf):
    try:
        sdf.take(1)
        return False
    except:
        return True

In [ ]:
def transitive_closure(graph_sdf, origins_sdf, depth):
    graph_sdf.cache()
    frontier_sdf = origins_sdf
    reachable_nodes_sdf = origins_sdf.cache()
    i = 0
    
    while ((not sdf_is_empty(frontier_sdf)) and (i < depth)):
        new_nodes_sdf = graph_sdf.join(frontier_sdf,graph_sdf.from_node == frontier_sdf.node).select('to_node').distinct()
        new_nodes_sdf = new_nodes_sdf.join(reachable_nodes_sdf, new_nodes_sdf.to_node == reachable_nodes_sdf.node,\
                                          'leftanti').withColumnRenamed('to_node', 'node')
        
        frontier_sdf = new_nodes_sdf.cache()
        reachable_nodes_sdf = reachable_nodes_sdf.union(new_nodes_sdf).cache()
        i += 1
        
    graph_sdf.unpersist()
    
    return reachable_nodes_sdf

In [ ]:
reachable_sdf = transitive_closure(graph_sdf, nodes_sdf, 3)

## Step 2.3 Results

In [1]:
reachable_sdf.count()

NameError: name 'reachable_sdf' is not defined

In [2]:
reachable_sdf.show()

NameError: name 'reachable_sdf' is not defined